In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
cleaning_data = pd.read_csv('Data/Tweet_Processed_DataCleaning_Done.csv')

In [3]:
cleaning_data.head()

,Tweet,Label
0,euedsonduarte lilovlog jairbolsonaro exatamen...,0.0
1,a china fecha o primeiro laboratorio do mundo ...,0.0
2,janeiro china mente sobre a de mortos nos caso...,0.0
3,nivel de poluicao na china cai drasticamente a...,0.0
4,eikebatista os que cruzam os oceanos trazem u...,0.0


## Bert

In [4]:
#!pip install transformers

### Preparing the train and test

In [5]:
# divinding in train, validation and text
x_data = cleaning_data['Tweet']
y_data = cleaning_data['Label']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=y_data, random_state=43)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=43)

### Tokenization

In [49]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

In [35]:
#pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [36]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [37]:
def preProcessing(data, maxLength, tokenizer):    
    for tweet in data:
        encoded = tokenizer.encode_plus(
                    tweet, #Sentence to be tokenized
                    add_special_tokens=True, #Adding [CLS] a token added to beggining of the setence
                                             #and [SEP] a token added to end of the setence
                    max_length=maxLength, #the max size of the setence
                    padding='max_length',#pad_to_max_length = True, #Adding [PAD] a token that represents the real sentence(when the setence is  
                                              #smaller than the max size the spaces will be completed with this token)                                              
                    return_attention_mask=True, #An array of 0 and 1 indicating which tokens are [PAD](space in blank) 
                                                #and the tokens belonging to the sentence
                    return_tensors = 'pt', #Return pytorch tensors(the same as numpy array)
        )
        
        
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    #return np.array(input_ids),np.array(attention_masks)

In [38]:
modelPath = "Model/BertBaseMultUncased"

In [39]:
#Loading locally the bert model
bert_tokenizer = BertTokenizer.from_pretrained(modelPath, local_files_only=True, do_lower_case=True)

PATH = 'models/cased_L-12_H-768_A-12/'
tokenizer = BertTokenizer.from_pretrained(PATH, local_files_only=True)

In [40]:
input_ids = []
attention_masks = []
# Tokenizing the senteces of the train dataset
preProcessing(x_train.values, 512, bert_tokenizer)
#torch.cat concatenate the tensors 
inputIdTrain = torch.cat(input_ids, dim=0)
attentionMaskTrain = torch.cat(attention_masks, dim=0)
labelsTrain = torch.tensor(y_train.values)

# Print sentence 0, now as a list of IDs.
#print('Original: ', x_train[0])
#print('Token IDs:', input_ids[0])

In [41]:
input_ids = []
attention_masks = []
# Tokenizing the senteces of the valid dataset
preProcessing(x_valid.values, 512, bert_tokenizer)
#torch.cat concatenate the tensors 
inputIdValid = torch.cat(input_ids, dim=0)
attentionMaskValid = torch.cat(attention_masks, dim=0)
labelsValid = torch.tensor(y_valid.values)

In [42]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [43]:
#TensorDataset encapsulates the data
train_data = TensorDataset(inputIdTrain, attentionMaskTrain, labelsTrain)
valid_data = TensorDataset(inputIdValid, attentionMaskValid, labelsValid)

In [44]:
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
# Dataloders wrap the data to offer a beter perfomance during the iteration
train_dataloader = DataLoader(
            train_data,  # The training samples.
            sampler = RandomSampler(train_data), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
valid_dataloader = DataLoader(
            valid_data, # The validation samples.
            sampler = SequentialSampler(valid_data), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [45]:
bertModel = BertForSequenceClassification.from_pretrained(modelPath, 
                                                          local_files_only=True, 
                                                          num_labels = 2, # The number of output labels--2 for binary classification.
                                                                          # You can increase this for multi-class tasks.   
                                                          output_attentions = False, # Whether the model returns attentions weights.
                                                          output_hidden_states = False,)

Some weights of the model checkpoint at Model/BertBaseMultUncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model check

### Maybe there's a problem to use BertForSequenceClassification for models that have support to Portuguese. 
### BertModel seems to work fine. It will be neeed for investigation, but to continue the development, i will change to BertModel.

# https://github.com/neuralmind-ai/portuguese-bert/issues/19

In [46]:
#To run the model in the GPU
bertModel.cuda();

In [47]:
params = list(bertModel.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (105879, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

##### Recommended value parameters(Bert Paper)
##### batch size: 16, 32
##### learning rate: 5e-5, 3e-5, 2e-5
##### epochs: 2,3,4

In [48]:
# AdamW optimizer reference https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
optimizer = AdamW(bertModel.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

C:\Users\Priscila\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [50]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [51]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [52]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))